In [ ]:
from datetime import datetime
import os
import pandas
import re

In [ ]:
EX_RUNTIME = 300 # seconds

In [ ]:
dir = os.scandir('raw/')
experiments = list(filter(lambda x: x.is_dir() and x.name[0] != '.', dir))


In [ ]:
def experiment_to_csv(path):
    jobs = pandas.read_csv(f'{experiment.path}/design.csv')
    jobs.columns.values[0] = 'Index'
    jobs = jobs[jobs.Index >= 0]
    
    jobs['accuracy'] = 0.0
    jobs['epochs'] = 0
    pat = re.compile("accuracy: (\d+\.\d+)")
    for idx, row in jobs.iterrows():
    #     {int(row.num_nodes)}
        row.failure_rate = 0.0
        row.failure_duration = 0.0
        filename = f"{int(row.Index)}-nodes{int(row.num_nodes)}-batch{int(row.batch_size)}-epochs50-frate{int(row.failure_rate)}-duration{int(row.failure_duration)}-time{EX_RUNTIME}.log"


        try:
            with open(f'{experiment.path}/{filename}', 'r') as file:
                content = file.read()
                result = pat.findall(content)
                jobs.at[idx, 'accuracy'] = result[-1] if len(result) > 0 else 0
                jobs.loc[idx, 'epochs'] = len(result)
        except FileNotFoundError as e:
            print(e)
            jobs.at[idx, 'accuracy'] = -1
            jobs.loc[idx, 'epochs'] = -1
    
    time = os.stat(f'{experiment.path}/design.csv').st_mtime
    dt = datetime.fromtimestamp(time)
    jobs.to_csv(f'ex-{dt.strftime("%Y-%m-%d_%H:%M")}.csv')
    jobs

In [ ]:
for experiment in experiments:
    if os.path.exists(f'{experiment.path}/design.csv'):
        experiment_to_csv(experiment.path)